In [8]:
## In-Class Assignment 4

import boto3
import pandas as pd

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## 1) Reading the csv file
heart = pd.read_csv(file_content_stream)

heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [9]:
## 2) Removing observations with missing values

heart = heart.dropna()

In [10]:
## 3) Using age, currentSmoker, totChol, BMI, and heartRate as the predictor variables, and TenYearCHD as the target variable 
## to split the data into two data-frames: train (80%) and test (20%).

from sklearn.model_selection import train_test_split

## Defining the input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [11]:
## 4) Using the train data-frame to build a logistic regression model

from sklearn.linear_model import LogisticRegression

## Building the logistic model
logit_md = LogisticRegression().fit(X_train, Y_train)

In [13]:
## 5) Using the model from part 4 to predict the likelihood of risk of coronary disease of the patients in the test data-frame. 
## Using 25% as the cutoff value

preds = logit_md.predict_proba(X_test)[:, 1]

import numpy as np

preds = np.where(preds < 0.25, 0, 1)

## Reporting the confusion matrix and the accuracy

from sklearn.metrics import accuracy_score, confusion_matrix

print('Confusion Matrix:\n', confusion_matrix(Y_test, preds))
print('/nAccuracy Score:', accuracy_score(Y_test, preds))

Confusion Matrix:
 [[545  82]
 [ 70  35]]
Accuracy Score: 0.7923497267759563
